## Import

In [69]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


### 1 - Télécharger le fichier alice.txt depuis l’espace de cours. Charger ce fichier en python et afficher son contenu.

In [70]:
alice_text = open("alice.txt", encoding='utf-8').read()

### 2 - Depuis la chaîne de caractères précédemment chargée, générer la liste de caractères uniques présents dans le livre. Écrire une méthode permettant de transformer chaque caratère en entier et inversement (chaque entier en caractère). Ceci est nécessaire car les réseaux de neurones ne travaillent pas sur des caractères mais sur des nombres !

In [71]:
unique_car = set()

for letter in alice_text:
    unique_car.add(letter)

In [72]:
car_to_int = {}
int_to_car = {}

for i, c in enumerate(unique_car):
    car_to_int[c] = i
    int_to_car[i] = c

### 3 - Préparer un dataset contenant des séquences de 50 caractères (paramétrable) de longueur en entrée et le caractère suivant cette séquence en target. 

In [73]:
def create_dataframe(data, decalage):
    X, y = [], []
    for i in range(len(data) - decalage):
        X.append(data[i:i+decalage])
        y.append(car_to_int[data[i+decalage]])
    return np.array(X), np.array(y)

In [74]:
decalage = 50
data_X, data_y = create_dataframe(alice_text, decalage)

In [75]:
df = pd.DataFrame({'x':data_X.tolist(), 'y':data_y})

In [76]:
df

,x,y
0,﻿The Project Gutenberg eBook of Alice’s Advent...,38
1,The Project Gutenberg eBook of Alice’s Adventu...,30
2,he Project Gutenberg eBook of Alice’s Adventur...,19
3,e Project Gutenberg eBook of Alice’s Adventure...,38
4,Project Gutenberg eBook of Alice’s Adventures...,89
...,...,...
163961,ibe to our email newsletter to hear about new ...,24
163962,be to our email newsletter to hear about new e...,53
163963,e to our email newsletter to hear about new eB...,4
163964,to our email newsletter to hear about new eBo...,68


### 4 - Transformer le dataset en séquence « one hot encodées »

In [77]:
x_encoded = []
for i, seq in enumerate(df['x']):
    x_encoded.append([])
    for letter in seq:
        x_encoded[i].append((car_to_int[letter]))
    

In [78]:
df['x'] =  x_encoded

In [79]:
df['x']

0         [9, 80, 6, 3, 38, 27, 14, 57, 37, 3, 35, 13, 3...
1         [80, 6, 3, 38, 27, 14, 57, 37, 3, 35, 13, 38, ...
2         [6, 3, 38, 27, 14, 57, 37, 3, 35, 13, 38, 31, ...
3         [3, 38, 27, 14, 57, 37, 3, 35, 13, 38, 31, 81,...
4         [38, 27, 14, 57, 37, 3, 35, 13, 38, 31, 81, 13...
                                ...                        
163961    [30, 66, 3, 38, 13, 57, 38, 57, 81, 14, 38, 3,...
163962    [66, 3, 38, 13, 57, 38, 57, 81, 14, 38, 3, 67,...
163963    [3, 38, 13, 57, 38, 57, 81, 14, 38, 3, 67, 60,...
163964    [38, 13, 57, 38, 57, 81, 14, 38, 3, 67, 60, 30...
163965    [13, 57, 38, 57, 81, 14, 38, 3, 67, 60, 30, 32...
Name: x, Length: 163966, dtype: object

In [80]:
n_total = len(df['x'])
N_seq = n_total - decalage
split_idx = int(N_seq * 0.7)

train_df = df.iloc[:split_idx]
test_df = df.iloc[split_idx:]

print(train_df.shape, test_df.shape)

(114741, 2) (49225, 2)


In [81]:
X_train = np.array(train_df['x'].tolist())
y_train = np.array(train_df['y'].tolist())

X_test = np.array(test_df['x'].tolist())
y_test = np.array(test_df['y'].tolist())

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(114741, 50) (114741,)
(49225, 50) (49225,)


### 5 - Proposer et tester différentes architecture LSTM entrainées sur le dataset précédemment généré. 

In [82]:
lstm = tf.keras.Sequential()

# Couche d'entrée de 50 neuronnes
lstm.add(tf.keras.Input(shape=(50,1)))
# Couche LSTM
lstm.add(tf.keras.layers.LSTM(50, return_sequences=True))
lstm.add(tf.keras.layers.LSTM(50, return_sequences=True))
lstm.add(tf.keras.layers.LSTM(50))
# Couche de sortir dense de 1 neuronne sans fonction d'activation
lstm.add(tf.keras.layers.Dense(1, activation=None))

adam = tf.keras.optimizers.Adam()
lstm.compile(loss='mean_squared_error', optimizer=adam)

In [ ]:
lstm.fit(X_train, y_train, epochs=10, verbose=1, batch_size=128)

Epoch 1/100
568/897 ━━━━━━━━━━━━━━━━━━━━ 20s 64ms/step - loss: 1075.3527

KeyboardInterrupt: 